<a href="https://colab.research.google.com/github/sugampath/practice/blob/master/Bertrawdata%2095%25%20Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

     |████████████████████████████████| 2.0MB 19.7MB/s 
     |████████████████████████████████| 3.2MB 53.1MB/s 
     |████████████████████████████████| 890kB 54.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=7272e158e27125fe857450ffb9a7f61a56ce5c5e57173c4c7e7a26b65b7cc4dc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup


In [4]:
data=pd.read_csv("/content/drive/MyDrive/Thesis/dataset.csv")

In [5]:
data.rename(columns = {"label":"category"}, inplace = True)

In [6]:
data.head(10)

,feature,category
0,सुदूरपश्चिम समाज कतारको दोस्रो साधारणसभा सम्पन...,diaspora
1,अमेरिकाबाट गायिका थापाको देशभक्ति गीत (भिडियो ...,diaspora
2,आदिवासी जनजाति मञ्च अष्ट्रेलियामा नयाँ कार्यसम...,diaspora
3,नेपाल फेस्टिवल मेलवर्नः नेपालका पर्यटनमन्त्री ...,diaspora
4,नेपाल फेष्टिबलको धन्यवाद ज्ञापन सम्पन्न\n\n१२ ...,diaspora
5,आगलागी पीडित थापालाई भेटे एनआरएन अध्यक्षले\n\n...,diaspora
6,फिजीका नेपालीलाई नेपाली भाषा पढाइने\nभीमसेन सा...,diaspora
7,अष्ट्रेलियामा नेपालीको पहिलो संस्थाले पायो नया...,diaspora
8,कराते खेलाडी ठकुरी ‘अमेरिका गट ट्यालेन्ट’मा जा...,diaspora
9,ईयूका लागि नेपाली राजदूत नियुक्त भएपछि थापाले ...,diaspora


In [7]:
data["feature"][996]

'सँयुक्त नेपाली मोर्चा युरोपको अध्यक्षमा डा. कृष्ण अर्याल सर्वसम्मत\n\n२० सेप्टेम्बर, पोर्चुगल । नयाँ नेतृत्व चयन गर्दै माओवादी केन्द्र निकट संयुक्त नेपाली मोर्चा युरोपको चौथों अधिबेशन सम्पन्न भएको छ ।\nअधिबेशनले नर्वे निवासी डा. कृष्ण अर्यालको अध्यक्षतामा आगामी दुई वर्षे कार्यकालका लागि २२ सदस्यीय नयाँ नेतृत्व सर्वसम्मत चयन गरेको हो ।\nजसको उपाध्यक्षहरुमा राममणि रिजाल (बेलायत), अच्युत फुयाल (स्विजरल्याण्ड) र कृष्ण पौडेल (बेल्जियम) रहेका छन् ।\nमोर्चाको महासचिबमा गंगाप्रसाद भुसाल (स्पेन) चयन भएका छन् । अधिबेशनले ३ सचिबमा मानबहादुर गुरुङ (बेलायत), कृष्णहरि कंडेल (स्पेन) र समिर अधिकारी (पोर्चुगल)लाई निर्बिरोध चयन गरेको छ ।\nयस्तै कोषाध्यक्षमा खड्ग बुढाथोकी (पोर्चुगल) रहेका छन् भने मोर्चाको प्रबक्तामा दर्शन आचार्य (स्विजरल्याण्ड)लाई चयन गरेको छ ।\nअधिबेशनले कार्यसमिति सदस्यहरुमा क्रमश प्रकाश पौडेल (बेल्जियम), बलमान ओली (नर्बे), कल्पना मगर (बेल्जियम), गंगा शर्मा (बेल्जियम), जितेन्द्र रायमाझी (चेक रिपब्लिक), महेश कुमार खाती (डेनमार्क), प्रकाश शर्मा (जर्मनी), सुशीला खतिवडा (बेल्जियम), निर्मल

In [8]:
data.shape

(19990, 2)

In [9]:
data["category"].value_counts()

health           2000
society          2000
economy          2000
entertainment    2000
sports           2000
politics         2000
technology       2000
opinion          2000
diaspora         2000
international    1990
Name: category, dtype: int64

In [10]:
possible_labels = data.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'diaspora': 0,
 'economy': 1,
 'entertainment': 2,
 'health': 3,
 'international': 4,
 'opinion': 5,
 'politics': 6,
 'society': 7,
 'sports': 8,
 'technology': 9}

In [11]:
possible_labels

array(['diaspora', 'economy', 'entertainment', 'health', 'international',
       'opinion', 'politics', 'society', 'sports', 'technology'],
      dtype=object)

In [12]:
data["label"]=data.category.replace(label_dict)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

In [15]:
data["data_type"]=["not_set"]*data.shape[0]
data.loc[X_train,"data_type"]="train"
data.loc[X_val,"data_type"]="Val"

In [16]:
data.groupby(["category","label","data_type"]).count()

feature
category      label data_type         
diaspora      0     Val            600
                    train         1400
economy       1     Val            600
                    train         1400
entertainment 2     Val            600
                    train         1400
health        3     Val            600
                    train         1400
international 4     Val            597
                    train         1393
opinion       5     Val            600
                    train         1400
politics      6     Val            600
                    train         1400
society       7     Val            600
                    train         1400
sports        8     Val            600
                    train         1400
technology    9     Val            600
                    train         1400

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [18]:
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].feature.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=510, 
    return_tensors='pt')


encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='Val'].feature.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=510, 
    return_tensors='pt')

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='Val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
a=tokenizer.tokenize(data["feature"][996])
print(a)
len(a)

['स', '##ँ', '##युक्त', 'नेपाली', 'म', '##ोर', '##्', '##चा', 'य', '##ुर', '##ोप', '##को', 'अध्यक्ष', '##मा', 'ड', '##ा', '.', 'कृष्ण', 'अ', '##र', '##्या', '##ल', 'सर्व', '##सम', '##्म', '##त', '२०', 'से', '##प', '##्ट', '##ेम्बर', ',', 'प', '##ोर', '##्', '##च', '##ुग', '##ल', '।', 'न', '##याँ', 'ने', '##त', '##ृत', '##्व', 'च', '##यन', 'ग', '##र', '##्द', '##ै', 'मा', '##ओ', '##वादी', 'केन्द्र', 'न', '##िक', '##ट', 'संयुक्त', 'नेपाली', 'म', '##ोर', '##्', '##चा', 'य', '##ुर', '##ोप', '##को', 'च', '##ौ', '##थ', '##ों', 'अ', '##धि', '##बे', '##शन', 'स', '##म्', '##पन', '##्न', 'भएको', 'छ', '।', 'अ', '##धि', '##बे', '##शनल', '##े', 'न', '##र', '##्वे', 'न', '##िव', '##ास', '##ी', 'ड', '##ा', '.', 'कृष्ण', 'अ', '##र', '##्या', '##ल', '##को', 'अध्यक्ष', '##ता', '##मा', 'आ', '##गा', '##मी', 'दु', '##ई', 'वर्ष', '##े', 'कार्य', '##काल', '##का', 'लागि', '२२', 'सदस्य', '##ीय', 'न', '##याँ', 'ने', '##त', '##ृत', '##्व', 'सर्व', '##सम', '##्म', '##त', 'च', '##यन', 'गरेको', 'हो', '।', 'ज', '##स

631

In [20]:
a=tokenizer.tokenize("सुदूरपश्चिम समाज कतारको साधारणसभा सम्पन्न कतार सुदूरपश्चिम सेवा समाज कतारको साधारणसभाले अमृत शाहीलाई कार्याकालका अध्यक्ष चयन बिबिध सांस्कृतिक कार्यक्रमसहित समाजको साधारणसभा सम्पन्न हिमाल पहाड तराई एकताको कामना समृद्ध समाजनिर्माण सुदुर पश्चिम सेवा समाजको चाहाना मूलनाराका बर्षअघि स्थापना संस्थाले बिभिन्न सामाजिक कामहरु गरिरहेको अध्यक्ष अमृत शाहीको सभापतित्वमा सम्पन्न कार्यक्रमको प्रमुख अतिथि सस्थाका संरक्षक समाजसेवी नरेन्ऽ भाट दोहाको नेशनल स्थित टपफम होटलको सभाहलमा सम्पन्न साधारणसभामा सुदूरपश्चिमका कलाकार गङ्गादत्त गिरी महेश नाथा रिमालामा नबराज प्यासी लोकेश बिसी प्रकाश केसी प्रकाश सिंह गौरब जैसी जोशी सिखालप्ली मनबहादुर जोरा लगायतले आ-आफ्नो प्रस्तुति राखेका संस्थाको आर्थिक प्रतिवेदन कोषाध्यक्ष उद्धब सिंहले प्रस्तुत सस्थाको तर्फबाट शुभकामना मन्तव्य बरिष्ठ उपाध्यक्ष सुरेन्द्र बिष्ट सनैया इकाइ कमिटी अध्यक्ष गणेश तिरुवाले प्रस्तुत कार्यक्रममा बैदेशिक रोजगारीको सिलसिलामा कतारमा सुदूरपश्चिमवासीको बाक्लो उपस्थिति कार्यक्रमका वक्ताहरुले संस्थाले अहिलेसम्म कामहरुको प्रशंसा आगामी कार्यसमितिलाई सफलताको शुभकामना दिएका साधारणसभाले सदस्यीय कार्यसमिति चयन कुमार अविरल पन्थीको सहयोगमा")
print(a)
len(a)
print(a[:250])

['स', '##ु', '##दू', '##र', '##प', '##श', '##्', '##च', '##िम', 'समाज', 'क', '##ता', '##रक', '##ो', 'स', '##ा', '##धारण', '##स', '##भा', 'स', '##म्', '##पन', '##्न', 'क', '##ता', '##र', 'स', '##ु', '##दू', '##र', '##प', '##श', '##्', '##च', '##िम', 'सेवा', 'समाज', 'क', '##ता', '##रक', '##ो', 'स', '##ा', '##धारण', '##स', '##भा', '##ले', 'अ', '##म', '##ृत', 'श', '##ाह', '##ील', '##ाई', 'कार्य', '##ा', '##काल', '##का', 'अध्यक्ष', 'च', '##यन', 'ब', '##ि', '##ब', '##ि', '##ध', 'सांस्कृतिक', 'कार्यक्रम', '##स', '##हित', 'समाज', '##को', 'स', '##ा', '##धारण', '##स', '##भा', 'स', '##म्', '##पन', '##्न', 'ह', '##िम', '##ाल', 'प', '##हा', '##ड', 'तर', '##ाई', 'एक', '##ता', '##को', 'काम', '##ना', 'स', '##म', '##ृ', '##द्ध', 'समाज', '##नि', '##र्मा', '##ण', 'स', '##ु', '##द', '##ुर', 'पश्चिम', 'सेवा', 'समाज', '##को', 'च', '##ाह', '##ाना', 'मूल', '##ना', '##रा', '##का', 'ब', '##र्ष', '##अ', '##घ', '##ि', 'स्थापना', 'संस्था', '##ले', 'ब', '##ि', '##भ', '##िन', '##्न', 'सामाजिक', 'काम', '##हर', '##ु',

In [21]:
encoded_data_train



{'input_ids': tensor([[   101,    898,  14070,  ...,    925,  20486,    102],
        [   101,  18263, 102116,  ...,  21835,    880,    102],
        [   101,  29953,    886,  ...,      0,      0,      0],
        ...,
        [   101,  98440,  13718,  ...,  40265,  11714,    102],
        [   101,    884,  69897,  ...,  50101,  11554,    102],
        [   101,    867,  78660,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [22]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [23]:
len(dataset_train),len(dataset_val)

(13993, 5997)

In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [25]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [26]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [27]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [28]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [29]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [31]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [32]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    FILE="model.pth"   
    torch.save(model.state_dict(),FILE)
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.6485400812095231
Validation loss: 0.4586260242506106
F1 Score (Weighted): 0.911790998231485



Epoch 2
Training loss: 0.33985537151030704
Validation loss: 0.3582393506667727
F1 Score (Weighted): 0.9393955410094458



Epoch 3
Training loss: 0.21393183616356448
Validation loss: 0.3457608129583664
F1 Score (Weighted): 0.9443582957041898



Epoch 4
Training loss: 0.14306120828000204
Validation loss: 0.3644432836616974
F1 Score (Weighted): 0.9468759801502474



Epoch 5
Training loss: 0.08197319698220154
Validation loss: 0.3408548194503606
F1 Score (Weighted): 0.9515635034930334



In [33]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [34]:
model.load_state_dict(torch.load('/content/model.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [35]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [36]:
accuracy_per_class(predictions, true_vals)

Class: diaspora
Accuracy: 561/600

Class: economy
Accuracy: 570/600

Class: entertainment
Accuracy: 562/600

Class: health
Accuracy: 553/600

Class: international
Accuracy: 574/597

Class: opinion
Accuracy: 559/600

Class: politics
Accuracy: 590/600

Class: society
Accuracy: 558/600

Class: sports
Accuracy: 588/600

Class: technology
Accuracy: 592/600

